<a href="https://colab.research.google.com/github/ShubhFuladi06/ANN-With-Batch-Norm-and-Transfer-LearningL/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# ROOT = "/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/FSDS_Feb_Batch/ALL_NOTES/Day-08"
# os.chdir(ROOT)

In [ ]:
# assert os.getcwd() == ROOT

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 8s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980 - 8s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.3851 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138 - 5s/epoch - 3ms/step
Epoch 9/

In [ ]:
model.save("pretrained_mnist_model.h5")

# Transfer learning 

## New Problem statement - 

Classify handwritten digits into odd and even

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

# New Section

In [ ]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [ ]:
for layer in pretrained_mnist_model.layers[:-1]: # leaves the last layer unfreezed => trainable
  layer.trainable = False # freezing the layers from getting trained
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [ ]:
# try:
#   del new_model
# except:
#   pass

In [ ]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

# LAYERS = [
#           lower_pretrained_layers,
#           tf.keras.layers.Dense(2, activation="softmax")
# ]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [ ]:
100 * 2 + 2

202

In [ ]:
266610 - 1010

265600

In [ ]:
265600 + 202

265802

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
9%2  ### odd data

1

In [ ]:
6%2

0

In [ ]:
np.where(y_train%2 == 0, 1, 0)

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
y_train[0], y_train[-1]

(7, 8)

In [ ]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels

In [ ]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [ ]:
np.unique(y_train_bin)

array([0, 1])

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676 - 5s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8754 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2884 - val_accuracy: 0.8832 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.2978 - accuracy: 0.8752 - val_loss: 0.2792 - val_accuracy: 0.8874 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890 - 5s/epoch - 3ms/step
Epoch 9/

In [ ]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 3ms/step - loss: 0.2768 - accuracy: 0.8861


[0.2768453359603882, 0.8860999941825867]

In [ ]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [ ]:
np.argmax(new_model.predict(X_new), axis=-1)


array([0, 1, 0])